In [36]:
import ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from ipynb.fs.full.csv_functions import write_to_csv, read_from_csv
from ipynb.fs.full.get_samples import get_data, read_data_of_one_subject, print_length_signal, plot_labels, get_samples_signal, get_samples
from ipynb.fs.full.ECG_features import get_ecgfeatures

In [37]:
def  plot_signal(signal, fs):
    t=np.arange(0,signal.size*(1/fs),(1/fs))
    t=t[:signal.size]
    
    plt.figure(figsize=(12,4))
    plt.plot(t,signal)
    plt.xlabel('$Time (s)$') 
    plt.ylabel('$signal$')

In [38]:
def main():
    df_ecgfeatures = pd.DataFrame(None, columns= ['feat_nn20', 'feat_nn50', 'feat_rmssd', 'feat_nnhr', 
                                     'feat_sdsd', 'feat_pnn50', 'feat_pnn20', 'peak vlf', 'peak lf','peak hf', 'norm power lf', 
                                     'norm power hf', 'power ratio', 'total power'])
    fs = 700
    y = []
    for i in range(2,19):
        if i != 12:
            subject = 'S' + str(i)
            print(subject)
            for label in range(1,5):
                print(label)
                if label==3 or label==4:
                    y.append(1)
                else:
                    y.append(label)
                data = get_data(subject)
                labels = data.get_labels()
                ecg_base, resp_base, eda_base = get_samples_signal(data, labels, label, fs)
                df_ecgfeatures = get_ecgfeatures(ecg_base, fs, subject, label, df_ecgfeatures)
    return df_ecgfeatures, y

In [39]:
# write_to_csv(df_ecgfeatures, 'out_features.csv')

In [40]:
# features = pd.concat([df_ecgfeatures, df_edafeatures], ignore_index=True)